In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    with open('sarcasm.json') as f:
        datas = json.load(f)

    sentences = []
    labels = []

    for data in datas:
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])
    
    train_sentences = sentences[:training_size]
    train_labels = labels[:training_size]

    validation_sentences = sentences[training_size:]
    validation_labels = labels[training_size:]

    tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(train_sentences)

    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

    train_padded = pad_sequences(train_sequences, maxlen = max_length, truncating = trunc_type, padding = padding_type)
    validation_padded = pad_sequences(validation_sequences, maxlen = max_length, truncating = trunc_type, padding = padding_type)

    train_labels = np.array(train_labels)
    validation_labels = np.array(validation_labels)

    model = Sequential([
                        Embedding(vocab_size, embedding_dim, input_length=max_length),
                        Bidirectional(LSTM(64, return_sequences=True)),
                        Bidirectional(LSTM(128, return_sequences=True)),
                        Bidirectional(LSTM(128, return_sequences=True)),
                        Bidirectional(LSTM(64)),
                        Dense(128, activation='relu'),
                        Dense(64, activation='relu'),
                        Dense(32, activation='relu'),
                        tf.keras.layers.Dense(1, activation='sigmoid')

    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    checkpoint_path = 'my_checkpoint_ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only = True,
                                 save_best_only = True,
                                 monitor = 'val_loss',
                                 verbose = 1)
    
    model.fit(train_padded, train_labels,
              validation_data = (validation_padded, validation_labels),
              callbacks = [checkpoint],
              epochs = 20)
    model.load_weights(checkpoint_path)

    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("TF4-sarcasm.h5")

Epoch 1/20
624/625 [============================>.] - ETA: 0s - loss: 0.4564 - acc: 0.7752
Epoch 1: val_loss improved from inf to 0.40732, saving model to my_checkpoint_ckpt
625/625 [==============================] - 42s 54ms/step - loss: 0.4565 - acc: 0.7750 - val_loss: 0.4073 - val_acc: 0.7973
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.3627 - acc: 0.8351
Epoch 2: val_loss improved from 0.40732 to 0.37299, saving model to my_checkpoint_ckpt
625/625 [==============================] - 31s 50ms/step - loss: 0.3627 - acc: 0.8351 - val_loss: 0.3730 - val_acc: 0.8289
Epoch 3/20
624/625 [============================>.] - ETA: 0s - loss: 0.3328 - acc: 0.8509
Epoch 3: val_loss did not improve from 0.37299
625/625 [==============================] - 31s 50ms/step - loss: 0.3328 - acc: 0.8508 - val_loss: 0.3735 - val_acc: 0.8307
Epoch 4/20
625/625 [==============================] - ETA: 0s - loss: 0.3131 - acc: 0.8612
Epoch 4: val_loss did not improve from 0.37299
625